### NOTE:
I was having issues with timing, even on the CSC servers so I often had to set a timeout limit. It is possible that for the LSH 
I used fnames[:n] where n is how many documents used. I need to review this code and see why there were issues with LSH otherwise when running all of the abstracts through it. 

I was also working with several notebooks on the servers and this is an amalgamation of the code used for the various steps.

However, I noticed there were several repeated abstracts,e.g. 'Note Available' and 'N/A' occurs over 5000 times in the data set (among other repeats, or near repeats). 

In hindsight I should have at least removed the duplicates as it would have been a more sane approach to testing the validity of the methods.  As there are at least 6000 duplicates (and more near duplicates) most of the pairs found could have been found using the collections.Counter() method. Which defeats the purpose of this exercise. 

In [9]:
import numpy 
import os 
import re 
import binascii 
from time import time
from os import listdir
import itertools
import pickle


In [2]:
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part1.zip
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part2.zip
#!wget https://archive.ics.uci.edu/ml/machine-learning-databases/nsfabs-mld/Part3.zip

In [2]:
import zipfile 
for f in listdir(): 
        if '.zip' in f:
            with zipfile.ZipFile(f, 'r') as zip_ref: 
                zip_ref.extractall() 

In [3]:
fnames = [] 
for root_loc in ['Part1/', 'Part 2/', 'Part 3/']:
    for loc in listdir(root_loc):
        for loc2 in listdir(root_loc+loc+'/'): 
            if 'html' not in loc2:
                for file in listdir(root_loc+loc+'/'+loc2+'/'):
                    if '.txt' in file: 
                        fnames.append(root_loc+loc+'/'+loc2+'/'+file)

In [4]:
def read_file(fname): 
    with open(fname, 'r', encoding='UTF-8') as f:
        # skip all lines until abstract 
        for line in f: 
            if "Abstract    :" in line:
                break

        # get abstract as a single string 
        abstract = ' '.join([line[:-1].strip() for line in f]) 
        abstract = re.sub(' +', ' ', abstract) # remove double spaces 
        return abstract
    
fname = fnames[130000] 
print(fname,read_file(fname))

Part 3/awards_2002/awd_2002_26/a0226848.txt 0226848 Ghazi This award will support the 3rd annual International Conference/Workshop on High Resolution Sector Field ICPMS to be held at the Department of Geology, Georgia State University, Atlanta, GA on October 2-5, 2002. This international conference is a continuation of the previous two successful meetings that were held in Norfolk, Virginia, (2000), and in Vienna, Austria (2001). There are now more than 300 single collector and nearly 100 multicollector high resolution ICP mass spectrometers around the world. We request funding to organize this meeting to bring together all the scientists around the world who work in the field of high resolution ICP mass spectrometry and accessory instrumentation (e.g., laser ablation, HPLC, CE, Microdrill techniques). This conference is intended to be a forum for the exchange of ideas and the presentation of work completed or in progress, for comment and discussion among international research groups 

In [5]:
error_messages, problem_files = [], []
 
for file in fnames: 
    try:
        read_file(file) 
    except Exception as e: 
            error_messages.append(e.reason)
            problem_files.append(file)
fnames = [file for file in fnames if file not in problem_files] 
print(f'{len(problem_files)} files had issues reading')

332 files had issues reading


In [6]:
set(error_messages), len(problem_files)

({'invalid continuation byte', 'invalid start byte'}, 332)

In [7]:
def get_shingles(fname, k=5):
    """Get all shingles from requested file (hashes of these shingles)
    """
    with open(fname, 'r') as f:
        # skip all lines until abstract
        for line in f:
            if "Abstract    :" in line:
                break

        # get abstract as a single string
        abstract = ' '.join([line[:-1].strip() for line in f])
        abstract = re.sub(' +', ' ', abstract)  # remove double spaces

        L = len(abstract)
        shingles = set()  # we use a set to automatically eliminate duplicates
        for i in range(L-k+1):
            shingle = abstract[i:i+k]
            crc = binascii.crc32(shingle.encode('utf-8')) #& 0xffffffff  # hash the shingle to a 32-bit integer
            shingles.add(crc)
        return shingles

In [8]:
fname = fnames[9]
print("file: {}".format(fname))
print("number of shingles: {}".format(len(get_shingles(fname, k=5))))
print("example of shingles: {}".format(list(get_shingles(fname, k=5))[:5]))

file: Part1/awards_1990/awd_1990_00/a9000050.txt
number of shingles: 337
example of shingles: [1598752772, 37695495, 502552604, 1814253598, 2860077100]


In [ ]:
shingles_vectors = []

for file in fnames: 
    sh = list(get_shingles(file, k=5))
    shingles_vectors.append(sh)

In [ ]:
def jaccard_similarity_score(x, y):
    """
    Jaccard Similarity J (A,B) = | Intersection (A,B) | /
                                    | Union (A,B) |
    """
    intersection_cardinality = len(set(x).intersection(set(y)))
    union_cardinality = len(set(x).union(set(y)))
    return intersection_cardinality / float(union_cardinality)

jaccard_similarity_score(shingles_vectors[0], shingles_vectors[1])

In [ ]:
def pairwise_comparison(fnames, s=0.9, k=5):
    t = time()
    candidates = []
    for pair in itertools.combinations(fnames,2):
        js = jaccard_similarity_score(get_shingles(pair[0], k=k),get_shingles(pair[1], k=k))

        if js > s:
            candidates.append(pair)
    t1 = time()-t
    return candidates, t1

In [ ]:
#pw_candidates5, pw_pairTime5 = pairwise_comparison(fnames, k=5)
#pw_candidates10, pw_pairTime10 = pairwise_comparison(fnames, k=10)

In [10]:
with open(r"pw_dict.pckl", "rb") as input_file:
     pw_dict = pickle.load(input_file)

In [11]:
for key in pw_dict:
    print(f'With {key} for pairwise comparison, {len(pw_dict[key][0])} pairs were found, which took {round(pw_dict[key][1])} seconds')

With k=5 for pairwise comparison, 15755 pairs were found, which took 14400 seconds
With k=10 for pairwise comparison, 4442 pairs were found, which took 14400 seconds


In this case I had to set a time limit as it kept going over the csc time limit. What can see in this case is that k=5 found for more candidate pairs than k=10. 

However, having exerimented with a smaller sample of the data k=10 is a bit slower than k=5.

In [ ]:
# set global parameters to process the whole dataset
bands = 5
rows = 5
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,))
B = numpy.random.randint(0, nextPrime, size=(nsig,))

In [ ]:
ShingleID = list(get_shingles(fname, k=5))[0]

print("random shingle: {}".format(ShingleID))

hashCode = ((A[0]*ShingleID + B[0]) % nextPrime) % maxShingleID
print("its hash code by first hash function: {}".format(hashCode))
hashCode = ((A[1]*ShingleID + B[1]) % nextPrime) % maxShingleID
print("its hash code by second hash function: {}".format(hashCode))

In [ ]:
# naive version of Minhash algorithm that computes a signature for a single file
# all shingles from that file are given in 'shingles'

def minhash(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = []
    for i in range(nsig):  # number of hash functions == nsig
        minHashCode = maxShingleID + 1
        a = A[i]
        b = B[i]
        
        for ShingleID in shingles:
            hashCode = ((a*ShingleID + b) % nextPrime) % maxShingleID
            if hashCode < minHashCode:
                minHashCode = hashCode

        signature.append(minHashCode)
    return signature

In [ ]:
# fast implementation of Minhash algorithm
# computes all random hash functions for a shingle at once, using vector operations
# also finds element-wise minimum of two vectors efficiently
def minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig):
    signature = numpy.ones((nsig,)) * (maxShingleID + 1)

    for ShingleID in shingles:
        hashCodes = ((A*ShingleID + B) % nextPrime) % maxShingleID
        numpy.minimum(signature, hashCodes, out=signature)

    return signature

In [ ]:
def cosine_distance(x, y):
    from scipy.spatial.distance import cosine
    similarity_score = 1-cosine(x, y)
    return similarity_score

In [ ]:
def run_min_hashing(fnames, s=0.9, k=5, cosine_dist=False):
    signatures = []  # signatures for all files
    for fname in fnames:
        shingles = get_shingles(fname, k=k)
        signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
        signatures.append(signature)

    Nfiles = len(signatures)
    t = time()
    candidates = []
    for i in range(Nfiles):
        for j in range(i+1, Nfiles):
            if cosine_dist:
                Jsim = cosine_distance(signatures[i], signatures[j])
            else:
                Jsim = numpy.mean(signatures[i] == signatures[j])  # average number of similar items in two vectors
            if Jsim >= s:
                candidates.append((i,j))
    t2 = time() - t

    return candidates, t2

In [ ]:
#mh_candidates_s5, mh_pairTime_s5 = run_min_hashing(fnames, s=0.5)
#mh_candidates_Cos, mh_pairTime_Cos = run_min_hashing(fnames, s=0.9, cosine_dist=True)
#mh_candidates_s95, mh_pairTime_s95 = run_min_hashing(fnames, s=0.95)

In [ ]:
#mh_candidates_s5, mh_pairTime_s5 = run_min_hashing(fnames, s=0.5)
#mh_candidates_s9, mh_pairTime_s9 = run_min_hashing(fnames, s=0.9)
#mh_candidates_s95, mh_pairTime_s95 = run_min_hashing(fnames, s=0.95)

In [ ]:
#mh_candidates_Cos, mh_pairTime_Cos = run_min_hashing(fnames, s=0.9, cosine_dist=True)


In [33]:
with open(r"minhash_dict_3hr.pckl", "rb") as input_file:
     mh_dict2 = pickle.load(input_file)
        
with open(r"minhash_dict1.pckl", "rb") as input_file:
     mh_dict1 = pickle.load(input_file)
        
with open(r"cosD.pckl", "rb") as input_file:
     mh_cosD = pickle.load(input_file)

In [22]:
for key in mh_dict1:
    print(f'With {key} for minhashing, {len(mh_dict1[key][0])} pairs were found, which took {round(mh_dict1[key][1])} seconds')

With k=5 for minhashing, 3798680 pairs were found, which took 14401 seconds
With k=10 for minhashing, 3879026 pairs were found, which took 14401 seconds


I had issues with timing so I used a time out of 4 hours for each. 

With K = 5 fewer pairs were found than for K=10. That could be because as with pairwise, k=10 is slower.

In [23]:
for key in mh_dict2:
    print(f'With {key} for minhashing, {len(mh_dict2[key][0])} pairs were found, which took {round(mh_dict2[key][1])} seconds')

With s=0.5 for minhashing, 3179659 pairs were found, which took 10800 seconds
With s=0.9 for minhashing, 3190341 pairs were found, which took 10801 seconds
With s=0.95 for minhashing, 3180771 pairs were found, which took 10800 seconds


Here I used a timeout of 3 hours for each (I ran the analysis for these 3 with a limit of 10 hours so this would give me time to save the results).

The number of pairs are misleading as the higher the _S_ threshold the fewer pairs should be found. This is because it is a stricter threshold. However, due to the timeout limit, this is not reflected in the numbers of pairs.

K=5 was used for all variations of S.

In [35]:
print(f'With k=5 for minhashing and Cosine distance (similarity), {len(mh_cosD[0])} pairs were found, which took {round(mh_cosD[1])} seconds')

With k=5 for minhashing and Cosine distance (similarity), 2861753 pairs were found, which took 28800 seconds


For the minhashing with cosine distance I calculated cosine distance and subtracted it from 1 (i.e. 1-cosine_dist). 286k pairs were found and it took 8 hours (my timeout limit for this case).

In [ ]:
def LSH(signatures, bands, rows, Ab, Bb, nextPrime, maxShingleID):
    """Locality Sensitive Hashing
    """
    numItems = signatures.shape[1]
    signBands = numpy.array_split(signatures, bands, axis=0)
    candidates = set()
    for nb in range(bands):
        hashTable = {}
        for ni in range(numItems):
            item = signBands[nb][:,ni]
            hash = (numpy.dot(Ab[nb,:], item) + Bb[nb]) % nextPrime % maxShingleID
            if hash not in hashTable:
                hashTable[hash] = [ni]
            else:
                hashTable[hash].append(ni)
        for _,items in hashTable.items():
            if len(items) > 1:
                L = len(items)
                for i in range(L-1):
                    for j in range(i+1, L):
                        cand = [items[i], items[j]]
                        numpy.sort(cand)
                        candidates.add(tuple(cand))
    return candidates

In [ ]:
def get_LSH(fnames, s=.9, k=5):

    # find candidates with LSH

    signatures = []  # signatures for all files
    for fname in fnames:
        shingles = get_shingles(fname, k=5)
        signature = minhash_vectorized(shingles, A, B, nextPrime, maxShingleID, nsig)
        signatures.append(signature)

    # prepare data for LSH
    A2 = numpy.random.randint(0, nextPrime/2, size=(bands, rows))  # now we need a vector of A parameters for each band
    B2 = numpy.random.randint(0, nextPrime/2, size=(bands, ))
    signatures = numpy.array(signatures).T  # LSH needs a matrix of signatures, not a list of vectors

    s = 0.95  # similarity threshold
    Nfiles = signatures.shape[1]  # number of different files
    t = time()
    candidates = LSH(signatures, bands, rows, A2, B2, nextPrime, maxShingleID)
    t2 = time() - t
    
    return candidates, t2

In [ ]:
#lsh_candidates25, lsh_pairTime25 = get_LSH(fnames, k=5)

In [ ]:
# set global parameters to process the whole dataset
bands = 10
rows = 10
nsig = bands*rows  # number of elements in signature, or the number of different random hash functions

maxShingleID = 2**32-1  # record the maximum shingle ID that we assigned
nextPrime = 4294967311  # next prime number after maxShingleID

A = numpy.random.randint(0, nextPrime, size=(nsig,))
B = numpy.random.randint(0, nextPrime, size=(nsig,))

In [ ]:
#lsh_candidates100, lsh_pairTime100 = get_LSH(fnames, k=5)


In [24]:
with open(r"lsh_dict_K.pckl", "rb") as input_file:
     lsh_dict_K = pickle.load(input_file)
        
with open(r"lsh_dict_100_2.pckl", "rb") as input_file:
     lsh_dict_100 = pickle.load(input_file)


In [29]:
for key in lsh_dict_K:
    print(f'With {key} and 25 rows & bands for LSH, {len(lsh_dict_K[key][0])} pairs were found, which took {round(lsh_dict_K[key][1])} seconds')

With k=5 and 25 rows & bands for LSH, 159049 pairs were found, which took 1256 seconds
With k=10 and 25 rows & bands for LSH, 159048 pairs were found, which took 855 seconds


Using 5 rows and 5 bands, k=5 took 1256 seconds and found 159049 pairs while k=10 was faster with 855 seconds.

In [30]:
print(f'With k=5 and 100 rows & bands for LSH, {len(lsh_dict_100[0])} pairs were found, which took {round(lsh_dict_100[1])} seconds')

With k=5 and 100 rows & bands for LSH, 13837748 pairs were found, which took 2476 seconds


Using k=5 and 10 rows and 10 bands, 13837748 pairs were found and it took 2476 seconds.

In [43]:
def find_1NN(candidates):
    cluster_dict = {}
    for pairs in list(candidates):
        p1, p2 = pairs
        if p1 not in cluster_dict:
            cluster_dict[p1]=[p1, p2]
        else:
            cluster_dict[p1]+=[p2]
    return cluster_dict    

For the 1-NN I will use k=5 for pairwise, minhashing and LSH. For LSH I will use 5x5 rows and bands. 

In [103]:
clusters_pw = find_1NN(pw_dict['k=5'][0])
clusters_mh = find_1NN(mh_dict2['s=0.95'][0])
clusters_lsh = find_1NN(lsh_dict_K['k=5'][0])

In [78]:
for key in clusters_pw:
    print(key, len(clusters_pw[key]))

Part1/awards_1990/awd_1990_00/a9000054.txt 2
Part1/awards_1990/awd_1990_00/a9000132.txt 2
Part1/awards_1990/awd_1990_00/a9000177.txt 3
Part1/awards_1990/awd_1990_00/a9000201.txt 2
Part1/awards_1990/awd_1990_00/a9000221.txt 5252
Part1/awards_1990/awd_1990_00/a9000222.txt 5251
Part1/awards_1990/awd_1990_00/a9000223.txt 5250


In [81]:
key = 'Part1/awards_1990/awd_1990_00/a9000221.txt' 
ls = clusters_pw[key]
for n in ls[:4]:
    print(n,read_file(n))
    print('\n'*2)

Part1/awards_1990/awd_1990_00/a9000221.txt Not Available



Part1/awards_1990/awd_1990_00/a9000222.txt Not Available



Part1/awards_1990/awd_1990_00/a9000223.txt Not Available



Part1/awards_1990/awd_1990_00/a9000396.txt Not Available





As I mentioned earlier there are a lot of texts that simply state 'Not available' (there are at least 5250ish). It seems that due to my timeout limit of 4 hours, only 7 texts were compared to all items. However, lets look at the ones that found fewer pairs.

In [83]:
key = 'Part1/awards_1990/awd_1990_00/a9000201.txt' 
ls = clusters_pw[key]
for n in ls:
    print(n,read_file(n))
    print('\n'*2)

Part1/awards_1990/awd_1990_00/a9000201.txt  The three classes of symbiont-bearing sarcodines (acantharia, polycystine radiolaria and foraminifera) are a conspicuous component of upper ocean communities. Most of the previous research on the ecology of these taxa has been concentrated on a single species or limited group of species. Dr. Caron and colleagues will simultaneously examine the ecological role of the entire assemblage of these three protozoa. Their research examines two hypotheses: 1. In nutrient-poor oceanic environments, the symbiont primary production in the entire population of acantharia, polycystine radiolaria and foraminifera will represent a significant fraction of the total primary production and will dominate the production by large cells (operationally defined as cells larger than 74um). 2. Sarcodines will be a disproportionately important component of sinking material from the euphotic zone compared with their suspended biomass. Symbiont production rates of the ent

Again, we appear to have found duplicates. Perhaps I should have removed the duplicates from the data set in order to properly test this method, though I suspect a would still have had timout issues. 

In [122]:
clusters_mh.keys()[]

TypeError: 'dict_keys' object is not subscriptable

In [123]:
for key in list(clusters_mh.keys())[:10]:
    print(key, len(clusters_mh[key]))

12 2
32 2
52 3
61 2
67 5252
68 5251
69 5250
85 2
106 45
111 2


In [113]:
for key in clusters_mh:
    ls = clusters_mh[key]

    text1 = read_file(fnames[ls[0]]).strip()
    text2 = read_file(fnames[ls[1]]).strip()
    if text1==text2 or key==32:
        continue
    else:
        break


In [120]:
text1, f'Length: {len(text1)}', text2 , f'Length: {len(text2)}'

('The three classes of symbiont-bearing sarcodines (acantharia, polycystine radiolaria and foraminifera) are a conspicuous component of upper ocean communities. Most of the previous research on the ecology of these taxa has been concentrated on a single species or limited group of species. Dr. Caron and colleagues will simultaneously examine the ecological role of the entire assemblage of these three protozoa. Their research examines two hypotheses: 1. In nutrient-poor oceanic environments, the symbiont primary production in the entire population of acantharia, polycystine radiolaria and foraminifera will represent a significant fraction of the total primary production and will dominate the production by large cells (operationally defined as cells larger than 74um). 2. Sarcodines will be a disproportionately important component of sinking material from the euphotic zone compared with their suspended biomass. Symbiont production rates of the entire population of symbiont-bearing sarcodi

By filtering out duplicates, it seems that our method also found near duplicates. 

In [124]:
for key in list(clusters_lsh.keys())[:10]:
    print(key, len(clusters_lsh[key]))

27062 40
67882 4
22807 14
16563 7
7487 163
19055 32
32154 2
20057 30
41472 35
51925 7


In [126]:
for key in clusters_lsh:
    ls = clusters_lsh[key]

    text1 = read_file(fnames[ls[0]]).strip()
    text2 = read_file(fnames[ls[1]]).strip()
    if text1==text2 or key==32:
        continue
    else:
        break


In [129]:
read_file(fnames[ls[2]]).strip(), text1, text2

('9312570 Vander This project will develop demonstrational, visual programming techniques and determine in which contexts programmers prefer them to textual programming techniques. An interactive, two-view environment will be provided in which users can create programs by manipulating concrete pictorial, examples of data structures or by entering code into a textual editor. Users will be able to create journal- quality pictures of data structures, such as arrays, lists, trees, and graphs, fill in example values. The system will use various inferencing strategies to construct appropriate general-purpose code. Users will be able to move back and forth between the visual and textual editors, using whichever representation is most natural. By providing both a visual and a textual representation, it should be possible to determine those operations which a programmer views as intrinsically textual. Ultimately these observations should allow better programming environments to be built that pr

LSH worked a seems to have worked a bit better, it found duplicates and near duplicates. However, we can see here that it also found a pairs of scientific papers.

Considering the number of pairs found and the number of duplicates I could simply do the following and gain a similar number pairs. This was my own stupidity. I have learnt my lesson and will make sure I inspect the data more efficiently before I start any analysis.

In [137]:
from collections import Counter

abstract = []
for file in fnames:
    abstract.append(read_file(file)[:200].strip())
    
c = Counter(abstract)
c.most_common(10)

[('Not Available', 5252),
 ('The Mathematical Sciences Postdoctoral Research Fellowships are awards to recent recipients of doctoral degrees in the mathematical sciences. These awards are a means of contributing to the future vit',
  137),
 ('This is a contract', 97),
 ('', 69),
 ("9394512 Williams This is an abstract. It doesn't matter what the margin settings are, nor will you have to worry about length or the three asterisks at the bottom. The new abstract load will take any",
  62),
 ('Postdoctoral Fellowship', 38),
 ('This is a contract.', 34),
 ('This award will support study of the Japanese language by an American scientist or engineer by providing a stipend, tuition, or other course-related expenses. The objectives of the program are to enab',
  28),
 ('Mathematical Sciences Fellowship', 24),
 ('This award is made under the innovative technology connections portion of NCRI\'s "Connections to the Internet" announcement, NSF 96-64, which covers K-12 education institutions (includ

I should have removed these duplicates before starting to more sanely compare the methods. By just looking at the documents with 'Not Available' would have found 13789126 pairs. Considering the timeouts I employed most of the comparisons found were these type of repeats. 